In [2]:
'''
build model. 

tarin model 

'''


import numpy as np
import pandas as pd
from data_process import preprocess, to_vector
# from data_generator import PairSequence
from spectrum_utils.spectrum import MsmsSpectrum
import sklearn.preprocessing
import re
import argparse
import tensorflow as tf
import data_generator
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import Input
from tensorflow.keras.callbacks import Callback, CSVLogger, ModelCheckpoint
from tensorflow.keras.layers import concatenate, Conv1D, Dense, Flatten, \
    Lambda, MaxPooling1D, Reshape
from tensorflow.keras.models import Model
from tensorflow_addons.optimizers import RectifiedAdam, AdamW
from tensorflow.keras import layers
import os
import pickle
from tensorflow.python.ops import math_ops




In [3]:
f = open("/home/workplace/ms2/embedding/LIBRARY_AUGMENT-adfc8252-download_filtered_mgf_library-main.mgf")
mgf = f.readlines()
f.close()

In [4]:
# negative_pairs = pd.read_csv("negative_pair.tsv", delimiter="\t")
# positive_pairs = pd.read_csv("positive_pair.tsv", delimiter="\t")
# negative_pairs = negative_pairs[:100000]
# positive_pairs = positive_pairs[:10000]
test_pos = pd.read_csv("test_pos_12_11.tsv", delimiter="\t")
test_neg = pd.read_csv("test_neg_12_11.tsv", delimiter="\t")
train_pos = pd.read_csv("train_pos_12_11.tsv", delimiter="\t")
train_neg = pd.read_csv("train_neg_12_11.tsv", delimiter="\t")

#change this code for test
test_pos = pd.read_csv("dataset/test_pos.tsv", delimiter="\t")
test_neg = pd.read_csv("dataset/test_neg.tsv", delimiter="\t")
train_pos = pd.read_csv("dataset/train_pos.tsv", delimiter="\t")
train_neg = pd.read_csv("dataset/train_neg.tsv", delimiter="\t")

In [5]:
# negative_pairs = negative_pairs.sample(frac=1).reset_index(drop=True)
# positive_pairs = positive_pairs.sample(frac=1).reset_index(drop=True)
# test_neg = negative_pairs[:negative_pairs.shape[0]//4]
# train_neg = negative_pairs[negative_pairs.shape[0]//4:].reset_index(drop=True)

# test_pos = positive_pairs[:positive_pairs.shape[0]//4]
# train_pos = positive_pairs[positive_pairs.shape[0]//4:].reset_index(drop=True)

# test_neg.to_csv("test_neg_12_11.tsv", sep="\t", index=False)
# train_neg.to_csv("train_neg_12_11.tsv", sep="\t", index=False)
# test_pos.to_csv("test_pos_12_11.tsv", sep="\t", index=False)
# train_pos.to_csv("train_pos_12_11.tsv", sep="\t", index=False)


In [6]:
train_neg

,sequence,spec_sequence,begin,end,pepmass
0,LDPNFLLK,LLVDMDPR,58795662,58795698,479.756946
1,IIQEPPHKK,C+57.021ADDRADLAK,231461791,231461912,568.291748
2,LYLGSGYFTAIQNLR,TELYQSLADLNNVR,59519854,59520155,818.422485
3,QAQELSWEK,YWVSPSPAK,215278600,215278655,517.770121
4,GIAASGHFIC+57.021VGTWSGR,MAPVPLDDSNRPASLTK,32871811,32871935,906.468550
...,...,...,...,...,...
4781380,MAANLYAR,VHGSPLELK,147770205,147770344,490.282386
4781381,ALDQYLMEFNAC+57.021R,LKYPSSPYSAHISK,188346860,188346888,789.420000
4781382,MGSPAPLLPGDLLGGGSDSI,FIQVC+57.021TQLQVLTEAFR,71115857,71116054,977.019612
4781383,C+57.021PAESVDSSGK,YLGAMQVADK,129830734,129830902,548.278869


In [7]:
def euclidean_distance(xy: list[tf.Tensor]):
    """
    Euclidean distance between two vectors using Keras.

    Parameters
    ----------
    xy : List[tf.Tensor]
        List of two vectors between which to compute the Euclidean distance.

    Returns
    -------
    The Euclidean distance between the two given vectors.
    """
    x, y = xy
    # sumsquare = x+y
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))
    # return sumsquare

def eucl_dist_output_shape(shapes):
    """
    Get the shape of the Euclidean distance output.

    Parameters
    ----------
    shapes
        Input shapes to the Euclidean distance calculation.

    Returns
    -------
    The shape of the Euclidean distance output.
    """
    shape1, shape2 = shapes
    return shape1[0], 1

def contrastive_loss(y_true: tf.Tensor, y_pred: tf.Tensor):
    """
    Contrastive loss function adapted from Hadsell et al. 2006.
    (http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf)

    The contrastive loss is modified so that it takes a certainty that labels
    are correct into account. This helps the neural network to overcome
    incorrectly labeled instances.

    Parameters
    ----------
    y_true : tf.Tensor
        The true class labels.
    y_pred : tf.Tensor
        The predicted embedded Euclidean distances.

    Returns
    -------
    The contrastive loss between the true and predicted class labels.
    """
    ramp_square = K.square(K.minimum(y_pred, 1))
    # ramp_square = K.log(y_pred)
    # margin_square = K.log(1-y_pred)
    margin_square = K.square(K.maximum(1 - y_pred, 0))
    y_true = tf.cast(y_true, tf.float32)
    return K.mean(y_true * 0.99 * ramp_square +
                  (1 - y_true * 0.99) * margin_square)
    
    
# def contrastive_loss(y_true:tf.Tensor, y_pred : tf.Tensor):
#     pass

def similarity(xy: list[tf.Tensor]):
    x, y = xy
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)
    
    

In [8]:
class Embedder:
    def __init__(self, num_fragment_features : int =2449, num_seq_features : int = 50, lr : float=0.01, filename : str = 'embedder.hdf5'):
        self.num_fragment_features = num_fragment_features
        self.num_seq_features = num_seq_features 
        self.filename = filename
        self.model = None
        self.lr = lr
        
    
    def build_fragment_embedder_model(self)->Model:
        filters = 30
        kernel_size = 3
        strides = 1
        pool_size = 1
        pool_strides = 2
        
        # seq_input = Input((self.num_seq_features, ), name='input_seq')
        # seq_dense32 = Dense(32, activation = 'selu', kernel_initializer='lecun_normal', name='seq_dense_32')(seq_input)
        # seq_dense200 = Dense(200, activation='selu', kernel_initializer='lecun_normal', name='seq_dense_200')(seq_dense32)
        
        
        fragment_input = Input((self.num_fragment_features,), name = 'input_fragment')
        fragment_input_reshape = Reshape((self.num_fragment_features,1), name = 'fragment_input_reshape')(fragment_input)
        fragment_layer = fragment_input_reshape
        
        fragment_layer = Conv1D(filters,kernel_size, strides=strides, activation = 'selu', name = 'fragment_layer_block1_conv1')(fragment_layer)
        fragment_layer = Conv1D(filters,kernel_size, strides=strides, activation = 'selu', name = 'fragment_layer_block1_conv2')(fragment_layer)
        fragment_layer = MaxPooling1D(pool_size, pool_strides, name='fragment_block1_pool1')(fragment_layer)
        
        #block2
        fragment_layer = Conv1D(filters*2, kernel_size, strides=strides, activation = 'selu', name = 'fragment_layer_block2_conv1')(fragment_layer)
        fragment_layer = Conv1D(filters*2, kernel_size, strides=strides, activation = 'selu', name = 'fragment_layer_block2_conv2')(fragment_layer)
        fragment_layer = MaxPooling1D(pool_size, pool_strides, name='fragment_block2_pool1')(fragment_layer)
        
        #block3
        fragment_layer = Conv1D(filters*4, kernel_size, strides=strides, activation = 'selu', name = 'fragment_layer_block3_conv1')(fragment_layer)
        fragment_layer = Conv1D(filters*4, kernel_size, strides=strides, activation = 'selu', name = 'fragment_layer_block3_conv2')(fragment_layer)
        fragment_layer = Conv1D(filters*4, kernel_size, strides=strides, activation = 'selu', name = 'fragment_layer_block3_conv3')(fragment_layer)
        fragment_layer = MaxPooling1D(pool_size, pool_strides, name='fragment_block3_pool1')(fragment_layer)
        
        #block4
        fragment_layer = Conv1D(filters*8, kernel_size, strides=strides, activation = 'selu', name = 'fragment_layer_block4_conv1')(fragment_layer)
        fragment_layer = Conv1D(filters*8, kernel_size, strides=strides, activation = 'selu', name = 'fragment_layer_block4_conv2')(fragment_layer)
        fragment_layer = Conv1D(filters*8, kernel_size, strides=strides, activation = 'selu', name = 'fragment_layer_block4_conv3')(fragment_layer)
        fragment_layer = MaxPooling1D(pool_size, pool_strides, name='fragment_block4_pool1')(fragment_layer)
        
        #block5
        fragment_layer = Conv1D(filters*8, kernel_size, strides=strides, activation = 'selu', name = 'fragment_layer_block5_conv1')(fragment_layer)
        fragment_layer = Conv1D(filters*8, kernel_size, strides=strides, activation = 'selu', name = 'fragment_layer_block5_conv2')(fragment_layer)
        fragment_layer = Conv1D(filters*8, kernel_size, strides=strides, activation = 'selu', name = 'fragment_layer_block5_conv3')(fragment_layer)
        fragment_layer = MaxPooling1D(pool_size, pool_strides, name='fragment_block5_pool1')(fragment_layer)
        
        fragment_output = Flatten(name='fragment_flatten')(fragment_layer)
        
        fragment_output = Dense(200, activation = 'selu', kernel_initializer = 'lecun_normal', activity_regularizer='l2', name='output')(fragment_output)
        
        return Model(inputs = [fragment_input], outputs = [fragment_output], name = 'frag_embedder')
     
    def build_seq_embedder_model(self)->Model:
        
        seq_input = Input((self.num_seq_features, 21, ), name='input_seq')
        seq_dense32 = Dense(32, activation = 'selu', kernel_initializer='lecun_normal', name='seq_dense_32')(seq_input)
        seq_flatten = Flatten(name='seq_flatten')(seq_dense32)
        seq_dense200 = Dense(200, activation='selu', kernel_initializer='lecun_normal', name='seq_dense_200')(seq_flatten)
        
        return Model(inputs = [seq_input], outputs = [seq_dense200], name = 'seq_embedder')

    
    def build_model(self)->None:
        fragment_embedder_model = self.build_fragment_embedder_model()
        input_ms2 = [Input((self.num_fragment_features,),name='input_ms2_fragment')]
        output_ms2 = fragment_embedder_model(input_ms2)
        
        seq_embedder_model = self.build_seq_embedder_model()
        input_seq =  [Input((self.num_seq_features,21,),name='input_seq')]
        output_seq = seq_embedder_model(input_seq)
        # distance = (Lambda(euclidean_distance, eucl_dist_output_shape,
        #                    name='embedding_euclidean_distance')
        #             ([output_ms2, output_seq]))
        
        distance = (Lambda(similarity, eucl_dist_output_shape,
                           name='embedding_similarity')
                    ([output_ms2, output_seq]))
        
        #print(f"fragmnet input shape = {input_ms2} seq_input shape = {input_seq} ")
        

        return Model(inputs=[input_ms2, input_seq], outputs=distance,
                     name='base_model')
        # distance = (Lambda(euclidean_distance, eucl_dist_output_shape,
        #                    name='embedding_euclidean_distance')
        #             ([output_ms2, output_seq]))

        # return Model(inputs=[*input_ms2, *input_seq], outputs=distance,
        #              name='base_model')
        # dot_product = tf.tensordot(output_ms2, output_seq, 1)
        # pass
    
    def build(self) -> None:
        """
        Build the Siamese model and compile it to optimize the contrastive loss
        using Adam.

        Both arms of the Siamese network will use the same embedder model, i.e.
        the weights are tied between both arms.

        The model will be parallelized over all available GPUs if applicable.
        """
        # Both arms of the Siamese network use the same model,
        # i.e. the weights are tied.
        # strategy = tf.distribute.MirroredStrategy()
        # with strategy.scope():
        #     self.model = self.build_model()
            # Train using Adam to optimize the contrastive loss.
            # self.model.compile(RectifiedAdam(self.lr),
            #                            contrastive_loss,
            #                            metrics=[keras.metrics.Accuracy()])
        self.model = self.build_model()
        
        def custom_acc(y_true, y_pred):
            threshold = 5
            differnece = tf.abs(tf.subtract(y_true, y_pred)) - threshold
            boolean_results = tf.where(differnece>0, True, False)
            return K.mean(math_ops.cast(boolean_results, K.floatx()))
                        
        acc = tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.02)
        
        # self.model.compile(AdamW(self.lr),
        #                                contrastive_loss,
        #                                metrics=[acc])
        loss = tf.keras.losses.BinaryCrossentropy(
            from_logits=True,
            label_smoothing=0.0,
            axis=-1,
            name='binary_crossentropy'
        )
        self.model.compile(AdamW(self.lr),loss)
        
        
    def train(self, train_data, validation_data):
        filename_log = 'history.log'
        callbacks = [ModelCheckpoint('log' + '.epoch{epoch:03d}'),
                     CSVLogger(filename_log)]
        # self.model.fit(
        #     train_data,train_target,
        #     epochs=10, batch_size = 64,
        #     validation_data=(validation_data,validation_target),callbacks=callbacks)
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="./cp.ckpt",
                                                 save_weights_only=True,
                                                 verbose=1)
        
        self.model.fit(
            train_data,
            epochs=5,
            validation_data=(validation_data),
            callbacks=[cp_callback])
    # def train(self, train_generator: data_generator.PairSequence,
    #           steps_per_epoch: int = None, num_epochs: int = 1,
    #           validators: list[data_generator.PairSequence] = None) -> None:
    #     """
    #     Train the Siamese model.

    #     Parameters
    #     ----------                                                                          
    #     train_generator : data_generator.PairSequence
    #         The training data generator.
    #     steps_per_epoch : int
    #          Total number of in each epoch. Useful to record the validation
    #          loss at specific intervals.
    #     num_epochs : int
    #         The number of epochs for which training occurs.
    #     validators : List[data_generator.PairSequence]
    #         The validation data generators.
    #     """
    #     if self.model is None:
    #         raise ValueError("The Siamese model hasn't been constructed yet")

    #     # filename, ext = os.path.splitext(self.filename)
    #     filename_log = 'history.log'
    #     callbacks = [ModelCheckpoint('log' + '.epoch{epoch:03d}'),
    #                  ValidationCallback(validators, filename_log),
    #                  CSVLogger(filename_log)]
    #     self.model.fit(
    #         train_generator, steps_per_epoch=steps_per_epoch,
    #         epochs=num_epochs, callbacks=callbacks)



In [9]:
emb = Embedder()
emb.build()
emb.model.summary()

Model: "base_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ms2_fragment (InputL  [(None, 2449)]               0         []                            
 ayer)                                                                                            
                                                                                                  
 input_seq (InputLayer)      [(None, 50, 21)]             0         []                            
                                                                                                  
 frag_embedder (Functional)  (None, 200)                  4487570   ['input_ms2_fragment[0][0]']  
                                                                                                  
 seq_embedder (Functional)   (None, 200)                  320904    ['input_seq[0][0]']  

In [10]:
train_generator = data_generator.gen(file_name_pos=train_pos, file_name_neg=train_neg, batch_size=256, mgf=mgf)
test_generator = data_generator.gen(file_name_pos=test_pos, file_name_neg=test_neg, batch_size=256, mgf=mgf)

len(train_generator)

11663

In [11]:
len(train_generator) * 256

2985728

In [12]:

# for a in train_generator:
#     print((type(a[0])))
#     print(len(a[0][0]))
#     print(len(a[0][1]))
#     print(len(a[1]))
#     print(len(a[2]))
#     print("               ")
#     # print(a[2])
#     pos = 0
#     neg = 0
#     for i,j in zip(a[2][0], a[2][1]):
#         if i == j:
#             pos += 1
#         else:
#             neg += 1
#     print(f"pos = {pos}")
#     print(f"neg = {neg}")
#     # print(a[1].shape)
#     # print(a[2].shape)
#     break

In [13]:
emb.train(train_generator, test_generator)


Epoch 1/5
11663/11663 [==============================] - ETA: 0s - loss: 0.7036
Epoch 1: saving model to ./cp.ckpt
11663/11663 [==============================] - 14350s 1s/step - loss: 0.7036 - val_loss: 0.6912
Epoch 2/5
11663/11663 [==============================] - ETA: 0s - loss: 0.6912
Epoch 2: saving model to ./cp.ckpt
11663/11663 [==============================] - 14250s 1s/step - loss: 0.6912 - val_loss: 0.6912
Epoch 3/5
11663/11663 [==============================] - ETA: 0s - loss: 0.6912
Epoch 3: saving model to ./cp.ckpt
11663/11663 [==============================] - 14288s 1s/step - loss: 0.6912 - val_loss: 0.6912
Epoch 4/5
11663/11663 [==============================] - ETA: 0s - loss: 0.6912
Epoch 4: saving model to ./cp.ckpt
11663/11663 [==============================] - 14263s 1s/step - loss: 0.6912 - val_loss: 0.6912
Epoch 5/5
11663/11663 [==============================] - ETA: 0s - loss: 0.6912
Epoch 5: saving model to ./cp.ckpt
11663/11663 [============================

In [14]:
emb.model.summary()

Model: "base_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ms2_fragment (InputL  [(None, 2449)]               0         []                            
 ayer)                                                                                            
                                                                                                  
 input_seq (InputLayer)      [(None, 50, 21)]             0         []                            
                                                                                                  
 frag_embedder (Functional)  (None, 200)                  4487570   ['input_ms2_fragment[0][0]']  
                                                                                                  
 seq_embedder (Functional)   (None, 200)                  320904    ['input_seq[0][0]']  

In [15]:
emb.model.save("model3_12_12.h5")

/home/workplace/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
data = pd.read_csv("data_processed.tsv", delimiter="\t")

In [17]:
data

,begin_index,end_idx,pepmass,charge,scan,sequence,score,fdr,protein
0,0,42,652.337341,3,1,+42.011AAAAAAALQAKSDEKAAVAGK,0.844429,0.0,sp|P36578|RL4_HUMAN
1,44,106,1278.664673,2,2,+42.011AAAAGGGGPGTAVGATGSGIAAAAAGLAVYR,19.629346,0.0,sp|Q92922|SMRC1_HUMAN
2,108,174,852.779275,3,3,+42.011AAAAGGGGPGTAVGATGSGIAAAAAGLAVYR,24.510680,0.0,sp|Q92922|SMRC1_HUMAN
3,176,240,639.830340,4,4,+42.011AAAAGGGGPGTAVGATGSGIAAAAAGLAVYR,8.557637,0.0,sp|Q92922|SMRC1_HUMAN
4,242,353,953.960000,2,5,+42.011AAAAVGAGHGAGGPGAASSSGGAR,19.697894,0.0,sp|Q96K37|S35E1_HUMAN
...,...,...,...,...,...,...,...,...,...
2140860,259456258,259456318,839.410000,3,2140861,YYSSPQDLPYPDPAIAQFSVQK,7.900134,0.0,sp|P34932|HSP74_HUMAN
2140861,259456320,259456401,922.898987,2,2140862,YYTSASGDEM+15.995VSLKDY,6.935526,0.0,sp|P07900|HS90A_HUMAN
2140862,259456403,259456490,883.424510,2,2140863,YYVLLHDVSAGDEQ+0.984R,9.998685,0.0,sp|Q9Y2L5|TPPC8_HUMAN
2140863,259456492,259456701,589.285732,3,2140864,YYVLLHDVSAGDEQ+0.984R,9.824301,0.0,sp|Q9Y2L5|TPPC8_HUMAN


In [18]:
cnt = 0
for a in data["sequence"]:
    for b in a:
        if b =="C":
            cnt += 1
            
print(cnt)

525929


In [19]:
cnt = 0
for a in data["sequence"]:
    for b in range(len(a)):
        if a[b] =="C":
            if a[b+1] == "+":
                cnt += 1
            
print(cnt)

525929


In [20]:
model = tf.keras.models.load_model('model3_11_27.h5')

OSError: No file or directory found at model3_11_27.h5